<a href="https://colab.research.google.com/github/Levelleor/googlecollab/blob/main/ComfyUIonColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#`mountUmount(`<font size="3px" color="#01c968">`Gdrive`</font>`)`



In [1]:
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Mount Gdrive to /content/drive</h3></center><br>
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive!
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

Mounted at /content/drive


#`Setup And Update ComfyUI`



In [2]:
from pathlib import Path

OPTIONS = {}

DRIVE_PATH = ""  # @param {type:"string"}
UPDATE_COMFY_UI = True  #@param {type:"boolean"}
WORKSPACE = '/content/ComfyUI'
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI

if DRIVE_PATH:

    WORKSPACE = DRIVE_PATH+"/ComfyUI"
    %cd {DRIVE_PATH}

![ ! -d WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI
%cd $WORKSPACE

if OPTIONS['UPDATE_COMFY_UI']:
  !echo -= Updating ComfyUI =-
  !git pull

!echo -= Install dependencies =-
!pip install xformers!=0.0.18 -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://download.pytorch.org/whl/cu118 --extra-index-url https://download.pytorch.org/whl/cu117

%cd {WORKSPACE}

-= Initial setup ComfyUI =-
Cloning into 'ComfyUI'...
remote: Enumerating objects: 29173, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 29173 (delta 47), reused 35 (delta 32), pack-reused 29101 (from 3)
Receiving objects: 100% (29173/29173), 75.44 MiB | 23.80 MiB/s, done.
Resolving deltas: 100% (19797/19797), done.
/content/ComfyUI
-= Updating ComfyUI =-
Already up to date.
-= Install dependencies =-
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu121, https://download.pytorch.org/whl/cu118, https://download.pytorch.org/whl/cu117
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.0/19.0 MB 124.4 MB/s 

# `Models Download`

## Download MODELS

In [8]:
# --- 0. Cleanup Workspace ---
!rm -rf /content/sample_data
%cd /content/ComfyUI/

# --- 1. System & Python Prerequisites ---
print("Installing Dependencies...")
!apt-get -y install -qq aria2

import os

# Detect if ComfyUI is using a venv and use that pip, otherwise use system pip
if os.path.exists('/content/ComfyUI/venv'):
    pip_cmd = '/content/ComfyUI/venv/bin/pip'
elif os.path.exists('/content/venv'):
    pip_cmd = '/content/venv/bin/pip'
else:
    pip_cmd = 'pip'

print(f"Targeting Python Environment: {pip_cmd}")

# Added 'addict' and 'yapf' to fix the Segment Anything / GroundingDINO error
!{pip_cmd} install segment-anything ultralytics scikit-image piexif timm transformers GitPython opencv-python-headless addict yapf

from google.colab import userdata

# --- 2. Setup API Token ---
try:
  CIVITAI_API_TOKEN = userdata.get('CIVITAI_API_TOKEN')
except:
  CIVITAI_API_TOKEN = None
print("Loaded API key:", "✅" if CIVITAI_API_TOKEN else "❌ Not found")

# --- 3. Helper Functions ---
def install_custom_node(url):
  %cd /content/ComfyUI/custom_nodes
  !git clone {url}

def downloadModel(url, filename=None):
  # Civitai Model Download Wrapper
  if filename:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{url}?token={CIVITAI_API_TOKEN}" -o "{filename}"
  else:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{url}?token={CIVITAI_API_TOKEN}"

# --- 4. Install Custom Nodes ---
print("Installing Custom Nodes...")
nodes = [
    'https://github.com/ltdrdata/ComfyUI-Manager.git',
    'https://github.com/ltdrdata/ComfyUI-Impact-Pack',
    'https://github.com/ltdrdata/ComfyUI-Impact-Subpack',
    'https://github.com/storyicon/comfyui_segment_anything',
    'https://github.com/Kosinkadink/ComfyUI-Advanced-ControlNet',
    'https://github.com/Fannovel16/comfyui_controlnet_aux',
    'https://github.com/Kosinkadink/ComfyUI-AnimateDiff-Evolved',
    'https://github.com/Kosinkadink/ComfyUI-VideoHelperSuite',
    'https://github.com/kijai/ComfyUI-KJNodes',
    'https://github.com/JPS-GER/ComfyUI_JPS-Nodes',
    'https://github.com/rgthree/rgthree-comfy',
    'https://github.com/cubiq/ComfyUI_IPAdapter_plus',
    'https://github.com/ssitu/ComfyUI_UltimateSDUpscale',
    'https://github.com/kijai/ComfyUI-Florence2',
    'https://github.com/kijai/ComfyUI-RMBG',
    'https://github.com/sipherxyz/comfyui-art-venture',
    'https://github.com/Kik00/ComfyUI-KikoTools',
    'https://github.com/1038lab/ComfyUI-RMBG'
]

for node in nodes:
    install_custom_node(node)

# --- 5. Download Civitai Models ---
print("Downloading Checkpoints...")
%cd /content/ComfyUI/models/checkpoints

civitai_checkpoints = [
    '2071650', # CyberRealistic Pony
    '2155386',
    '1759168',
    '1966530',
    '2442439',
    '2514955'
]

for model_id in civitai_checkpoints:
    downloadModel(f'https://civitai.com/api/download/models/{model_id}')

# --- 6. Download Civitai LoRAs ---
print("Downloading LoRAs...")
%cd /content/ComfyUI/models/loras
# dnd2 speed lora
downloadModel('https://civitai.com/api/download/models/1820705')

%cd /content/ComfyUI/
print("Main installation complete.")

/content/ComfyUI
Installing Dependencies...
Targeting Python Environment: pip
Loaded API key: ✅
Installing Custom Nodes...
/content/ComfyUI/custom_nodes
fatal: destination path 'ComfyUI-Manager' already exists and is not an empty directory.
/content/ComfyUI/custom_nodes
fatal: destination path 'ComfyUI-Impact-Pack' already exists and is not an empty directory.
/content/ComfyUI/custom_nodes
fatal: destination path 'ComfyUI-Impact-Subpack' already exists and is not an empty directory.
/content/ComfyUI/custom_nodes
fatal: destination path 'comfyui_segment_anything' already exists and is not an empty directory.
/content/ComfyUI/custom_nodes
fatal: destination path 'ComfyUI-Advanced-ControlNet' already exists and is not an empty directory.
/content/ComfyUI/custom_nodes
fatal: destination path 'comfyui_controlnet_aux' already exists and is not an empty directory.
/content/ComfyUI/custom_nodes
fatal: destination path 'ComfyUI-AnimateDiff-Evolved' already exists and is not an empty directory.


## OTHERS

In [4]:
# --- Download Public HuggingFace Models ---

# 1. VAE
print("Downloading VAE...")
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M -d /content/ComfyUI/models/vae -o "vae-ft-mse-840000-ema-pruned.safetensors" "https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors"

# 2. Xinsir ControlNet (Promax)
print("Downloading Xinsir ControlNet...")
# Create directory structure
!mkdir -p /content/ComfyUI/models/controlnet/SDXL/controlnet-union-sdxl-1.0/
# Download and Rename
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M -d /content/ComfyUI/models/controlnet/SDXL/controlnet-union-sdxl-1.0/ -o "SDXL_all_in_one_model_promax.safetensors" "https://huggingface.co/xinsir/controlnet-union-sdxl-1.0/resolve/main/diffusion_pytorch_model_promax.safetensors"

# 3. Upscaler
print("Downloading Upscaler...")
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M -d /content/ComfyUI/models/upscale_models "https://huggingface.co/shubhdotai/upscaler/resolve/9b9bfba2a119bee0d175427cadb05283a3ed2fb1/4xNMKDSuperscale_4xNMKDSuperscale.pt"

print("Public models downloaded.")


Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
cce944|OK  |   174MiB/s|/content/ComfyUI/models/vae/vae-ft-mse-840000-ema-pruned.safetensors

Status Legend:
(OK):download completed.

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
de7de0|OK  |   166MiB/s|/content/ComfyUI/models/controlnet/SDXL/controlnet-union-sdxl-1.0//SDXL_all_in_one_model_promax.safetensors

Status Legend:
(OK):download completed.

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
45e7d9|OK  |   127MiB/s|/content/ComfyUI/models/upscale_models/5a0bf058b451b95d7e102fd96939bb8632eb118db48e178b02dd186ba70cdef3

Status Legend:
(OK):download completed.
Public models downloaded.


## LIST MODELS

In [5]:
!ls -al ./models/checkpoints/

total 46062020
drwxr-xr-x  2 root root        4096 Dec 30 03:24 .
drwxr-xr-x 23 root root        4096 Dec 30 03:14 ..
-rw-r--r--  1 root root  6938040682 Dec 30 03:20 cyberrealisticPony_v127Alt.safetensors
-rw-r--r--  1 root root  6938041144 Dec 30 03:24 epicrealismXL_pureFix.safetensors
-rw-r--r--  1 root root  6938070306 Dec 30 03:22 jibMixRealisticXL_v180SkinSupreme.safetensors
-rw-r--r--  1 root root  7105350162 Dec 30 03:21 juggernautXL_ragnarokBy.safetensors
-rw-r--r--  1 root root  6938099634 Dec 30 03:20 lustifySDXLNSFW_ggwpV7.safetensors
-rw-r--r--  1 root root           0 Dec 30 03:14 put_checkpoints_here
-rw-r--r--  1 root root 12309866400 Dec 30 03:23 zImageTurbo_turbo.safetensors


# `START ComfyUI  & Expose Server (MANUAL)`

## Download Prerequisits

In [ ]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

In [ ]:
!python main.py --dont-print-server

# `START ComfyUI & Expose Server`

## Download Prerequisits

In [6]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

--2025-12-30 03:24:39--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64.deb [following]
--2025-12-30 03:24:40--  https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64.deb
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/8a32f7c6-649c-4f0d-806d-e14c19d0786d?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-30T04%3A06%3A11Z&rscd=attachment%3B+filename%3Dcloudflared-linux-amd64.deb&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4d

## CF Tunnel

In [ ]:
import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
    #print(l, end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server

[START] Security scan
[DONE] Security scan
## ComfyUI-Manager: installing dependencies done.
** ComfyUI startup time: 2025-12-30 03:49:26.522
** Platform: Linux
** Python version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
** Python executable: /usr/bin/python3
** ComfyUI Path: /content/ComfyUI
** ComfyUI Base Folder Path: /content/ComfyUI
** User directory: /content/ComfyUI/user
** ComfyUI-Manager config path: /content/ComfyUI/user/__manager/config.ini
** Log path: /content/ComfyUI/user/comfyui.log

Prestartup times for custom nodes:
   0.0 seconds: /content/ComfyUI/custom_nodes/rgthree-comfy
   5.5 seconds: /content/ComfyUI/custom_nodes/ComfyUI-Manager

Checkpoint files will always be loaded safely.
Total VRAM 15095 MB, total RAM 12976 MB
pytorch version: 2.9.1+cu128
xformers version: 0.0.33.post2
Set vram state to: NORMAL_VRAM
Device: cuda:0 Tesla T4 : cudaMallocAsync
Using async weight offloading with 2 streams
Enabled pinned memory 12326.0
working around nvidia conv3d memo

## localtunnel

In [ ]:
# localtunnel
!npm install -g localtunnel

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch localtunnel (if it gets stuck here localtunnel is having issues)\n")

  print("The password/enpoint ip for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
  p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
  for line in p.stdout:
    print(line.decode(), end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()
%cd /content/ComfyUI
!python main.py --dont-print-server